# Data Augmentation

In [ ]:
import os
import json
from datasets import load_from_disk
import pandas as pd

In [ ]:
base_dir = '/data/ephemeral/home/data/'
path_train = os.path.join(base_dir, 'train_dataset')
train_dic = load_from_disk(path_train)
print(train_dic)

In [ ]:
train_df = pd.DataFrame.from_dict(train_dic['train'])

## EDA

In [ ]:
import random
import nltk
from nltk.corpus import wordnet
from multiprocessing import Pool
from tqdm import tqdm

In [ ]:
nltk.download('punkt_tab')

In [ ]:
# 동의어 치환 함수
def synonym_replacement(words, n=1):
    new_words = words.copy()
    word_count = 0
    for i, word in enumerate(words):
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words[i] = synonym
            word_count += 1
        if word_count >= n:
            break
    return new_words

# 랜덤 삽입 함수
def random_insertion(words, n=1):
    for _ in range(n):
        new_word = random.choice(words)
        synonyms = wordnet.synsets(new_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name().replace('_', ' ')
            random_index = random.randint(0, len(words)-1)
            words.insert(random_index, synonym)
    return words

# 문장 내 단어 위치 바꾸기 함수
def random_swap(words, n=1):
    length = len(words)
    for _ in range(n):
        idx1, idx2 = random.sample(range(length), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# 문장 내 단어 삭제 함수
def random_deletion(words, p=0.5):
    if len(words) == 0:
        return words
    new_words = [word for word in words if random.uniform(0, 1) > p]
    return new_words if len(new_words) > 0 else [random.choice(words)]

# EDA 증강 기법을 위한 함수 (각 문장에 대해 증강 처리)
def eda_single_row(row, aug_idx):
    context_words = nltk.word_tokenize(row['context'])
    question_words = nltk.word_tokenize(row['question'])

    context_aug = random.choice([synonym_replacement(context_words),
                                 random_insertion(context_words),
                                 random_swap(context_words),
                                 random_deletion(context_words)])

    question_aug = random.choice([synonym_replacement(question_words),
                                  random_insertion(question_words),
                                  random_swap(question_words),
                                  random_deletion(question_words)])
    
    # context에서 answers의 text를 찾아 새로운 answer_start 업데이트
    answer_text = row['answers']['text'][0] # 정답이 여러 개일 수 있으므로 첫번째 정답 사용
    new_context = ' '.join(context_aug)
    new_answer_start = new_context.find(answer_text) # 새로운 context에서 정답 위치 찾기

    # answer_start가 존재하지 않으면 기본값 0으로 설정
    if new_answer_start == -1:
        new_answer_start = 0

    return {
        'title': row['title'],
        'context': new_context,
        'question': ' '.join(question_aug),
        'id': f"{row['id']}-aug-{aug_idx}",
        'answers': {
            'answer_start': [new_answer_start],
            'text' : [answer_text]
        },
        'document_id': row['document_id'],
        '__index_level_0__': row['__index_level_0__']
    }

# 멀티프로세싱을 사용한 EDA 처리
def eda(df, n_augmented=1):
    augmented_rows = []
    
    # tqdm으로 진행 상황 표시
    with Pool() as pool:
        for _, row in tqdm(df.iterrows(), total=len(df)):
            # augmented_rows += pool.map(eda_single_row, [row] * n_augmented)
            # 각 row에 대해 n_augmented 만큼 증강된 데이터를 생성, aug_idx를 0부터 시작
            results = pool.starmap(eda_single_row, [(row, aug_idx) for aug_idx in range(n_augmented)])
            augmented_rows.extend(results)
    
    augmented_df = pd.DataFrame(augmented_rows)
    concat_df = pd.concat([df, augmented_df], ignore_index=True)
    return concat_df


In [ ]:
augmented_train_df = eda(train_df, n_augmented=1)
augmented_train_df

In [ ]:
augmented_train_df.to_csv('/data/ephemeral/home/data/aug_eda.csv', index=False)

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기
    df = pd.read_csv(csv_file_path)
    
    # DataFrame을 JSON 파일로 저장
    df.to_json(json_file_path, orient='records', lines=True, force_ascii=False)

# 사용 예시
csv_file_path = '/data/ephemeral/home/data/aug_eda.csv'  # 변환할 CSV 파일 경로
json_file_path = '/data/ephemeral/home/data/aug_eda.json'  # 생성할 JSON 파일 경로

csv_to_json(csv_file_path, json_file_path)

print(f"'{csv_file_path}' 파일이 '{json_file_path}'로 변환되었습니다.")

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split

# 1. aug_eda.json 파일 로드
dataset = load_dataset('json', data_files='/data/ephemeral/home/data/aug_eda.json')

# 2. Hugging Face dataset을 pandas DataFrame으로 변환
df = dataset['train'].to_pandas()

# 3. train/validation으로 split
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# 4. pandas DataFrame을 다시 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data, preserve_index=False)
val_dataset = Dataset.from_pandas(val_data, preserve_index=False)

# 5. DatasetDict 형식으로 묶기
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# 6. train과 validation 데이터셋 저장 폴더 설정
output_dir = "/data/ephemeral/home/data/aug_train_dataset"
dataset_dict.save_to_disk(output_dir)

# 7. 저장된 데이터셋 불러오기
from datasets import load_from_disk
loaded_dataset_dict = load_from_disk(output_dir)

# 8. 불러온 데이터셋 확인
print(loaded_dataset_dict)